In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from google.colab import drive
from tqdm import tqdm
import dill
import numpy as np

In [ ]:
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/embeddings_dict.pkl', 'rb') as pkl:
    embeddings_dict = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/ai_word_bank_list.pkl', 'rb') as pkl:
    ai_word_bank_list = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/topic_dict.pkl', 'rb') as pkl:
    topic_dict = dill.load(pkl)

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"

In [ ]:
USE_model = hub.load(module_url)

In [ ]:
def most_similar(phrase: str) -> str :
    phrase_embedding = USE_model([phrase])
    #current_word_embedding = ""
    most_similar_value = 0
    most_similar_string = ""
    for word in ai_word_bank_list:
        if word == phrase:
            continue
        if np.inner(phrase_embedding, embeddings_dict[word]) > most_similar_value:
            most_similar_value = np.inner(phrase_embedding, embeddings_dict[word])
            most_similar_string = word
    return most_similar_string, most_similar_value

In [ ]:
most_similar("smoothie")

In [ ]:
def ai_player_guess(phrase: str) -> dict :
    smoothie_dict = {}
    dict_element_counter = 0
    phrase_embedding = USE_model([phrase])
    #current_word_embedding = ""
    minimum_similar_value = 0
    minimum_similar_string = ""
    for word in ai_word_bank_list:
        if word == phrase:
            continue
        if dict_element_counter < 15:
            dict_element_counter += 1
            smoothie_dict[word] = np.inner(phrase_embedding, embeddings_dict[word])  
            minimum_similar_string = min(smoothie_dict, key=smoothie_dict.get)
            #print(minimum_similar_string)
            #print(smoothie_dict[word])
            minimum_similar_value = smoothie_dict[minimum_similar_string]            
        
        elif np.inner(phrase_embedding, embeddings_dict[word]) > minimum_similar_value:
            smoothie_dict.pop(minimum_similar_string)
            smoothie_dict[word] = np.inner(phrase_embedding, embeddings_dict[word])
            minimum_similar_string = min(smoothie_dict, key=smoothie_dict.get)
            minimum_similar_value = smoothie_dict[minimum_similar_string]
          
    return smoothie_dict

In [ ]:
ai_player_guess('pudding')

In [ ]:
def build_similarity_dict (topic_dict: dict) -> dict :
    similarity_dict = {}
    for topic in topic_dict:
        similarity_dict[topic] = {}
        topic_literal = str(topic)
        for word in tqdm(ai_word_bank_list):
            if topic == word:
                continue
            else:
                #print(topic)
                similarity_dict[topic][word] = np.inner(embeddings_dict[word],
                                                        embeddings_dict[topic])
        for secret_word in topic_dict[topic]:
            similarity_dict[secret_word] = {}
            for element in tqdm(ai_word_bank_list):
                if secret_word == word:
                    continue
                else:
                    #print(type(secret_word))
                    #print(secret_word)
                    #print(type(element))
                    #print(element)
                    #print(type(embeddings_dict[secret_word]))

                    #similarity_dict[secret_word][element] = "fart"

                    similarity_dict[secret_word][element] = \
                    np.inner(embeddings_dict[secret_word],
                             embeddings_dict[element])

                    #np.inner(embeddings_dict[secret_word], embeddings_dict[element])
    return similarity_dict

In [ ]:
embeddings_dict['Math']

In [ ]:
similarity_dict = build_similarity_dict(topic_dict)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/similarity_dict.pkl', 'wb') as pkl:
    dill.dump(similarity_dict, pkl)

In [ ]:
listicle.append(topic_dict)

In [ ]:
type(listicle[0])

In [ ]:
rooms_dict = {'bah': 1231231, "ram": 34242, "ewe": 231414}
super_dict = {"Rooms": rooms_dict}

In [ ]:
super_dict[ ]

In [ ]:
outer_dictionary[1] = inner_dictionary

In [ ]:
outer_dictionary[1][1]

In [ ]:
chumba = dict()
wumba = "wumba"

In [ ]:
chumba[wumba] = {}

In [ ]:
chumba[wumba][wumba] = wumba

In [ ]:
chumba

In [ ]:
chumba[wumba] 